In [ ]:
#import the libraries and JSON files
from ftfy import fix_encoding
import pandas
import numpy
import json
import re

nist = json.load(open('2021-05-06_NIST consolidated.json'))
bigg = json.load(open('BiGG model of S. aureus.json'))
unknown_compounds = pandas.read_csv(open('2021-05-05_APF_unknown compounds to modelSEED_01.csv'))
empty_cell = ['nan', None, numpy.nan, '', ' ', 'NaN', '[]']


# creating a complete list of enzymatic reaction names in the BiGG model
saureus_enzymes = []
for key, value in bigg.items():
    #print(key)
    if key == 'reactions':
        for enzyme in value:
            for key2, value2 in enzyme.items():
                if key2 == 'name':
                    saureus_enzymes.append(value2)
                    
                    
# determining the set of unknown compounds, which originates from previous parsing executions
unknown_compounds_list = []
for compound in unknown_compounds['Unknown compound names']:
    if compound not in unknown_compounds_list:
        try:
            compound = fix_encoding(compound)
        except:
            pass
        
        unknown_compounds_list.append(compound)
        
#print(unknown_compounds_list)
                    

# compiling the set of reaction names from the NIST database
nist_enzyme_name = []
parenthesized_nist_enzymes = []
parenthesized_nist_ennzymes_original = []
and_conjoined_enzymes = []
and_conjoined_enzyme_original = []
thermodynamic_enzyme_names = []
kinetic_enzymes = []
for name, value in nist.items():
    nist_enzyme_name.append(name)
    for detail, information in value.items():
        #print(detail, '\t', information)
        if detail == 'Keq':
            for detail2, information2 in information.items():
                if detail2 == 'keqs':
                    for information3 in information2:
                        #print(id, '\t', information3)
                        if information3 not in empty_cell: 
                            #print('keqs', id)
                            if re.search('(\sand\s)', name):
                                left_enzyme_striped_value = re.sub('(.*and\s)', '', name)
                                right_enzyme_striped_value = re.sub('(\sand.*)', '', name)
                                if left_enzyme_striped_value not in thermodynamic_enzyme_names:
                                    thermodynamic_enzyme_names.append(left_enzyme_striped_value)
                                if right_enzyme_striped_value not in thermodynamic_enzyme_names:
                                    thermodynamic_enzyme_names.append(right_enzyme_striped_value)
                                if left_enzyme_striped_value not in and_conjoined_enzymes:
                                    and_conjoined_enzymes.append(left_enzyme_striped_value)
                                if right_enzyme_striped_value not in and_conjoined_enzymes:
                                    and_conjoined_enzymes.append(right_enzyme_striped_value)
                                if name not in and_conjoined_enzyme_original:
                                    and_conjoined_enzyme_original.append(name)
                            else:
                                if name not in thermodynamic_enzyme_names:
                                    thermodynamic_enzyme_names.append(name)
                        else:
                            print(information3)

        if detail == 'Enthalpy':
            for detail2, information2 in information.items():
                #print('yay')
                #print('information:\t', detail2, '\t', information2)
                if detail2 == 'enthalpy values':
                    #print('yes')
                    for information3 in information2:
                        if information3 not in empty_cell:
                            if re.search('(\sand\s)', name):
                                left_enzyme_striped_value = re.sub('(.*and\s)', '', name)
                                right_enzyme_striped_value = re.sub('(\sand.*)', '', name)
                                if left_enzyme_striped_value not in thermodynamic_enzyme_names:
                                    thermodynamic_enzyme_names.append(left_enzyme_striped_value)
                                if right_enzyme_striped_value not in thermodynamic_enzyme_names:
                                    thermodynamic_enzyme_names.append(right_enzyme_striped_value)    
                                if left_enzyme_striped_value not in and_conjoined_enzymes:
                                    and_conjoined_enzymes.append(left_enzyme_striped_value)
                                if right_enzyme_striped_value not in and_conjoined_enzymes:
                                    and_conjoined_enzymes.append(right_enzyme_striped_value)
                                if name not in and_conjoined_enzyme_original:
                                    and_conjoined_enzyme_original.append(name)
                            else:
                                if name not in thermodynamic_enzyme_names:
                                    thermodynamic_enzyme_names.append(name)
                        else:
                             print(information3)   
     
        if detail == 'Km':
            for detail2, information2 in information.items():
                #print('yay')
                #print('information:\t', detail2, '\t', information2)
                if detail2 == 'km values':
                    #print('yes')
                    for information3 in information2:
                        if information3 not in empty_cell:
                            if name not in kinetic_enzymes:
                                kinetic_enzymes.append(name)
                                #print('kinetics', name)
                        else:
                             print(information3)           
                                
        
        if re.search('(\s\(.*\))', name):
            striped_value = re.sub('(\s\(.*\))', '', name)
            #print(striped_value)
            parenthesized_nist_enzymes.append(striped_value)
            parenthesized_nist_ennzymes_original.append(name)
            
                
'''for enzyme in and_conjoined_enzyme_original:
    index = and_conjoined_enzyme_original.index(enzyme)
    for entry in nist[enzyme]['Keq']['keqs']:
        if entry not in empty_cell and and_conjoined_enzymes[index] not in thermodynamic_enzyme_names:
            thermodynamic_enzyme_names.append(enzyme)
    for entry in nist[enzyme]['Enthalpy']['enthalpy values']:
        if entry not in empty_cell and and_conjoined_enzymes[index] not in thermodynamic_enzyme_names:
            thermodynamic_enzyme_names.append(enzyme)'''
            
                
'''print('separated enzymes: %s\n' %(len(and_conjoined_enzymes)), set(and_conjoined_enzymes))   
print('\noriginal enzymes: %s\n' %(len(and_conjoined_enzyme_original)), set(and_conjoined_enzyme_original))     '''       

'''print(parenthesized_nist_enzymes)
nist_enzyme_name += and_conjoined_enzymes
thermodynamic_enzyme_names.append('test')'''
        
    
# comparing the NIST and BiGG databases 
represented_enzymes = []
represented_nist_enzymes = []
for enzyme in saureus_enzymes:
    for enzyme2 in thermodynamic_enzyme_names:
        if enzyme.lower() == enzyme2.lower():
            represented_enzymes.append(enzyme)
            represented_nist_enzymes.append(enzyme2)
    
    
'''    for enzyme2 in nist_enzyme_name:
        if enzyme.lower() == enzyme2.lower(): 
            if enzyme not in represented_enzymes:
                represented_enzymes.append(enzyme)
                represented_nist_enzymes.append(enzyme2)
    for enzyme3 in parenthesized_nist_enzymes:
        if re.search(enzyme3, enzyme):
            #print(enzyme)
            if enzyme not in represented_enzymes:
                index = parenthesized_nist_enzymes.index(enzyme3)
                represented_enzymes.append(enzyme)
                represented_nist_enzymes.append(parenthesized_nist_ennzymes_original[index])
    for enzyme3 in and_conjoined_enzymes:
        if re.search(enzyme3, enzyme):
            #print(enzyme)
            if enzyme not in represented_enzymes:
                print(enzyme, '\t', enzyme3)
                index = and_conjoined_enzymes.index(enzyme3)
                represented_enzymes.append(enzyme)
                represented_nist_enzymes.append(and_conjoined_enzyme_original[index])'''
                
                
'''unexpressed_enzymes = (set(nist_enzyme_name) - set(represented_nist_enzymes))
unexpressed_nist_enzymes_counted = len(nist_enzyme_name) - len(represented_enzymes)
not_thermodynamic_enzymes = set(nist_enzyme_name) - set(thermodynamic_enzyme_names)
    
print('\nTotal BiGG enzymes:', len(saureus_enzymes))
print('\nNot thermodynamic enzymes:', len(not_thermodynamic_enzymes))
print('\nTotal unexpressed NIST enzymes:', unexpressed_nist_enzymes_counted)
print('\nUnexpressed enzymes from the NIST database: %s\n' %(len(unexpressed_enzymes)), unexpressed_enzymes)
print('\nThermodynamic enzymes in the NIST database: %s\n' %(len(thermodynamic_enzyme_names)), 
      thermodynamic_enzyme_names)
print('\nS. aureus represented by the NIST database: %s\n' %(len(represented_enzymes)), represented_enzymes)
print('\nKinetics in the NIST database: %s\n' %(len(kinetic_enzymes)), kinetic_enzymes)'''

'''thermodynamic_dictionary = {}
for enzyme in thermodynamic_enzyme_names:
    thermodynamic_dictionary[enzyme] = '''


# export the parsed reaction names to a CSV file
'''dataframe = pandas.DataFrame.from_dict({'separated enzymes':thermodynamic_enzyme_names,
                                     'conjoined enzymes':and_conjoined_enzyme_original},
                            orient = 'index')
dataframe = dataframe.transpose()
dataframe.to_csv('2021-04-29_APF_thermodynamically described enzymes_01.csv')'''



# determine the set of metabolites in the thermodynamically described reactions
described_compounds = []
absent_compounds_from_modelseed = {}
for enzyme in nist:
    for enzyme2 in thermodynamic_enzyme_names:
        printed_enzyme = 'no'
        if re.search('(-)', enzyme2):
            enzyme2 = re.sub('(-)', '(\-)', enzyme2)
        if re.search(enzyme2, enzyme):
            #print(enzyme,'\n', enzyme2)
            reactants_list = []
            products_list = []
            for reaction in nist[enzyme]['reaction']:
                #print(reaction)
                reaction = fix_encoding(reaction)
                #print(reaction)
                if re.search('(=)', reaction):
                    reaction_split = reaction.split('=')
                    #print('reaction:\t', reaction)
                    reactants = reaction_split[0]
                    products = reaction_split[1] 
                    #print('reactants:\t', reactants)
                    #print('products:\t', products)
                    reactants_list = reactants.split(' + ')
                    products_list = products.split(' + ')
                    #print('reactants list:\t', reactants_list)
                    #print('products list:\t', products_list)
                    for element in reactants_list:
                        #print(element)
                        element = element.strip()
                        element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                        element = re.sub('(\d\s|\d\/\d\s)', '', element)
                        if element not in described_compounds:
                            #print(element)
                            described_compounds.append(element)
                        if element in unknown_compounds_list:
                            absent_compounds_from_modelseed[element] = {'Enzyme': enzyme,
                                                                        'Reaction': reaction,
                                                                        'synonym': ''}
                            
                            printed_enzyme = 'yes'
                            
                    for element in products_list:
                        #print(element)
                        element = element.strip()
                        element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                        element = re.sub('(\d\s|\d\/\d\s)', '', element)
                        if element not in described_compounds:
                            print(reaction)
                            print(element)
                            described_compounds.append(element)
                        if element in unknown_compounds_list and printed_enzyme == 'no' :
                            absent_compounds_from_modelseed[element] = {'Enzyme': enzyme,
                                                                        'Reaction': reaction,
                                                                        'synonym': ''}
                            
                            
                else:
                    print('abnormal reaction:\t', reaction)

'''with open('2021-05-09_APF_absent NIST compounds from modelSEED_01.json', 'w') as output:
    json.dump(absent_compounds_from_modelseed, output, indent = 4)'''
                    
                    

'''export_comopunds_yes_no = {}
for compound in described_compounds:
    export_comopunds_yes_no[compound] = {'logical name': 'yes',
                                        'present in KEGG': ''}

with open('2021-04-29_APF_nist thermodynamically described compounds for ModelSEED_02.json', 'w') as output:
    json.dump(export_comopunds_yes_no, output, indent = 4)'''


# create the clean set of reactions for the ModelSEED repository code
'''rearranged_reactions_for_modelseed = []
to_be_rearranged = []
export_reactions_for_modelseed = {}
for enzyme in nist:
    for enzyme2 in thermodynamic_enzyme_names:
        if re.search('(-)', enzyme2):
            enzyme2 = re.sub('(\-)', '\-', enzyme2)
        if re.search(enzyme2, enzyme):
            for reaction in nist[enzyme]['reaction']:
                reaction = fix_encoding(reaction)
                #print(reaction)
                if reaction not in to_be_rearranged:
                    to_be_rearranged.append(reaction)
                    #print(reaction)
                    
                    # replace the phase identifiers
                    if re.search('(\(aq\)|\(l\)|\(sln\)|\(liq\))', reaction):
                        reaction = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', reaction)
                    #print(reaction)
                    
                    # arrange the reaction equilibrium arrow and stoichiometric coefficients for ModelSEED
                    if re.search('(\s?=\s?)', reaction):
                        reaction = re.sub('(\s?=\s?)', ' <=> ', reaction)
                        
                    if re.search('((?<!\w)\d\s|\d\/\d\s)', reaction):
                        reaction = re.sub('((?<!\w)\d\s|\d\/\d\s)', r'(\1) ', reaction)
                        reaction = re.sub('(?<=\d)(\s)(?=\))', '', reaction)
                    else:
                        reaction = re.sub('(?<=\+)(\s)', ' (1) ', reaction)
                        reaction = re.sub('(?<=>)(\s)', ' (1) ', reaction)
                        
                    if reaction[0] == ' ':
                        reaction = re.sub('\s', '(1) ', reaction, 1)
                    else:
                        reaction = '(1) ' + reaction
                        
                    reaction = re.sub('(?<=\w)(\s)(?=<|\+)', '[0] ', reaction)
                    reaction += '[0]'
                    #print(reaction)
                    if reaction not in rearranged_reactions_for_modelseed:
                        enzyme2 = re.sub(r'(\\)', '', enzyme2)
                        export_reactions_for_modelseed[enzyme2] = {'reaction expression':reaction,
                                                                   'yes_no':''}
                        rearranged_reactions_for_modelseed.append(reaction)
                    else:
                        print('repeated reactions:\t', reaction)


print('\nFinal reactions:')
for reaction in rearranged_reactions_for_modelseed:
    print(reaction)
    
with open('2021-05-09_APF_nist thermodynamically described reactions for ModelSEED_01.json', 'w') as output:
    json.dump(export_reactions_for_modelseed, output, indent = 4)
'''

In [ ]:
import json
import re

wckb = json.load(open('2021-03-25_APF_WCKB reactions + references.json'))
wckb_metabolites = json.load(open('2021-03-17_APF_Karr et al. metabolite data.json'))

empty_cells = ['NaN', None, numpy.nan, '', ' ', 'null', 'None', 'nan', '[None]', '[\'None\']']    

# determine the set of thermodynamically described reactions from the WCKB downloaded json data
thermodynamically_described_enzymes_wckb = []
wckb_enzymes = []
wckb_keqs = []
wckb_kinetics = []
wckb_gibbs = []
for enzyme, value in wckb.items():
    if enzyme not in wckb_enzymes:
        #print(enzyme)
        wckb_enzymes.append(enzyme)
        for keq in value['keq']:
            if keq not in empty_cells and enzyme not in wckb_gibbs:
                thermodynamically_described_enzymes_wckb.append(enzyme)
                wckb_keqs.append(enzyme)
        for gibbs in value['Gibbs free energy']:
            if gibbs not in empty_cells and enzyme not in wckb_gibbs:
                thermodynamically_described_enzymes_wckb.append(enzyme)
                wckb_gibbs.append(enzyme)
        for forward_kinetics in value['forward kinetics']:
            if forward_kinetics not in empty_cells and enzyme not in wckb_gibbs:
                wckb_kinetics.append(enzyme)
        for backward_kinetics in value['backward kinetics']:
            if backward_kinetics not in empty_cells and enzyme not in wckb_gibbs:
                wckb_kinetics.append(enzyme)
                
print('\n\nWCKB enzymes:\t\t', len(wckb_enzymes)) 
print('Thermodynamically described enzymes: ', len(thermodynamically_described_enzymes)) 
print('WCKB Keqs:\t\t', len(wckb_keqs))   
print('WCKB Kinetics:\t\t', len(wckb_kinetics))
print('WCKB gibbs:\t\t', len(wckb_gibbs))     
                            
thermodynamically_described_enzymes = set(wckb_gibbs).union(set(wckb_keqs))
wckb_reactions = []
export_reactions_wckb_for_modelseed = {}
for enzyme in thermodynamically_described_enzymes:
    for entry in wckb[enzyme]['stoichiometry']:
        compound_loop = 0
        product_loop = 0
        for molecule in entry:
            if molecule['compartment'] == 'c':
                compartment_value = 0
            elif molecule['compartment'] == 'e':
                compartment_value = 1
            elif molecule['compartment'] == 'm':
                compartment_value = 2
                
            if compound_loop == 0:
                reaction_string = '(%s) %s[%d]'  %(abs(int(float(molecule['coefficient']))), molecule['molecule'], compartment_value)
                compound_loop += 1
            elif product_loop == 0 and float(molecule['coefficient']) > 0: 
                reaction_string += ' <=> (%s) %s[%d]'  %(int(float(molecule['coefficient'])), molecule['molecule'], compartment_value)            
                product_loop += 1
            else:
                reaction_string += ' + (%s) %s[%d]'  %(abs(int(float(molecule['coefficient']))), molecule['molecule'], compartment_value)
        print(reaction_string)
        wckb_reactions.append(reaction_string)       
        export_reactions_wckb_for_modelseed[enzyme] = {'reaction expression':reaction_string,
                                                       'yes_no':''}
                    
          
                
with open('2021-04-29_APF_wckb thermodynamically described reactions for ModelSEED_01.json', 'w') as output:
    json.dump(export_reactions_wckb_for_modelseed, output, indent = 4)
    
    
# determine the set of thermodynamically described compounds from the aforementioned reactions
'''described_compounds = []
for enzyme in thermodynamically_described_enzymes:
    for entry in wckb[enzyme]['stoichiometry']:
        for molecule in entry:
            if molecule['molecule'] not in described_compounds:
                described_compounds.append(molecule['molecule'])
            try:
                proper_molecule = wckb_metabolites[molecule['molecule']]['Name']
                #print(molecule['molecule'])
                #print(proper_molecule)
            except:
                print('abbreviation-only compound:\t', molecule['molecule'])
                proper_molecule = molecule['molecule']
                proper_molecule = re.sub('(MG_287_MONOMER.)', '', proper_molecule)
                proper_molecule = re.sub('(MG_287_MONOMER|NA)', 'unknown', proper_molecule)
            if proper_molecule not in described_compounds:
                described_compounds.append(proper_molecule)
            
print('\n\nThermodynamically described compounds: ', len(described_compounds))
export_wckb_compounds_yes_no = {}
for compound in described_compounds:
    export_wckb_compounds_yes_no[compound] = ''
    print(compound)
    
with open('2021-04-29_APF_wckb thermodynamically described compounds for ModelSEED_01.json', 'w') as output:
    json.dump(export_wckb_compounds_yes_no, output, indent = 4)'''


In [ ]:
import json
import re

wckb = json.load(open('2021-04-29_APF_nist thermodynamically described compounds for ModelSEED_02.json'))
nist = json.load(open('2021-04-29_APF_wckb thermodynamically described compounds for ModelSEED_01.json'))
wckb_2 = json.load(open('2021-04-29_APF_wckb thermodynamically described reactions for ModelSEED_01.json'))
nist_2 = json.load(open('2021-04-29_APF_nist thermodynamically described reactions for ModelSEED_02.json'))
    
    
# combine the thermodynamically described compounds from the NIST and WCKB sources
'''export_union_compounds = {}
union_compounds = []
excluded_compounds = []
nist_compounds = []
wckb_compounds = []
for compound in nist:
    nist_compounds.append(compound)
    for compound2 in wckb:
        wckb_compounds.append(compound2)
        if compound.lower() == compound2.lower():
            if compound not in union_compounds:
                union_compounds.append(compound)
                export_union_compounds[compound] = ""

print('union compounds:\t', len(union_compounds))
for compound in union_compounds:
    print(compound)
    
complete_compounds_set = set(nist_compounds).union(set(wckb_compounds))
sorted_set_compounds = sorted(complete_compounds_set)
complete_thermodynamic_compounds = {}
print('\n\ncomplete compounds set:\t', len(complete_compounds_set))
for compound in sorted_set_compounds:
    complete_thermodynamic_compounds[compound] = ''
    print(compound)
    
    
with open('2021-04-29_APF_complete thermodynamically described compounds for KEGG_01.json', 'w') as output:
    json.dump(complete_thermodynamic_compounds, output, indent = 4)
    
with open('2021-04-30_APF_ModelSEED formatted compounds.tsv', 'w') as output:
    #output.write("id" + "name" + "abbreviation" + "aliases" + "formula" + "mass" + "charge" + "deltag" + "deltagerr" + "pka" + "pkb" + "inchikey" + "smiles" + "is_cofactor" + "is_core" + "is_obsolete" + "abstract_compound" + "comprised_of" + "linked_compound" + "notes" + "source" + '\n')
    output.write('\t'.join(['NAMES','ID', '\n']))
    for compound in complete_thermodynamic_compounds:
        output.write('\t'.join([compound, ]) + '\n')'''
    
    
# combine the thermodynamically described reactions from the NIST and WCKB sources
'''export_union_reactions = {}
union_reactions = []
nist_reactions = []
wckb_reactions = []
for reaction in nist_2:
    nist_reactions.append(reaction)
    for reaction2 in wckb_2:
        wckb_reactions.append(reaction2)
        if reaction.lower() == reaction2.lower():
            if reaction not in union_reactions:
                union_reactions.append(reaction)
                export_union_reactions[reaction] =''
                
print('union reactions:\t', len(union_reactions))
for reaction in union_reactions:
    print(reaction)
    
complete_reactions_set = set(nist_reactions).union(set(wckb_reactions))
sorted_set_reactions = sorted(complete_reactions_set)
complete_thermodynamic_reactions = {}
print('\n\ncomplete compounds set:\t', len(sorted_set_reactions))
for reaction in sorted_set_reactions:
    #reaction = re.sub('(\))', '\)', reaction)
    #print(reaction)
    try:
        complete_thermodynamic_reactions[reaction] = {'reaction expression':nist_2[reaction]['reaction expression'],
                                                      'yes_no':''}
    except:
        complete_thermodynamic_reactions[reaction] = {'reaction expression':wckb_2[reaction]['reaction expression'],
                                                      'yes_no':''}
    
    
with open('2021-04-30_APF_complete thermodynamically described reactions for KEGG_01.json', 'w') as output:
    json.dump(complete_thermodynamic_reactions, output, indent = 4)
    
with open('2021-04-30_APF_ModelSEED formatted reactions.tsv', 'w') as output:
    output.write('\t'.join(['NAMES','reaction', '\n']))
    for reaction in complete_thermodynamic_reactions:
        output.write('\t'.join([reaction, 
                                complete_thermodynamic_reactions[reaction]['reaction expression']]) + '\n')'''

In [ ]:
import json
from ftfy import fix_encoding
import pandas

processing_data = json.load(open('2021-04-29_APF_nist thermodynamically described compounds for ModelSEED_01.json'))

illogically_named_enzymes = []
for compound in processing_data:
    #print(compound, '\n', processing_data[compound])
    if processing_data[compound]['logical name'] == 'no':
        compound = fix_encoding(compound)
        illogically_named_enzymes.append(compound)
        print(compound)
        
dataframe = pandas.DataFrame(data = {'Unknown compound names':illogically_named_enzymes})

print('\nTotal described compounds: ', len(illogically_named_enzymes))


with open('2021-05-05_APF_unknown compounds to modelSEED_01.csv', 'w', encoding="utf-8") as output:
    dataframe.to_csv(output)

In [ ]:
# import the libraries and the JSON data files        
import pandas
import numpy
import json
import re

nist = json.load(open('2021-03-29_APF_NIST consolidated_01.json'))
wckb_json = json.load(open('2021-03-25_APF_WCKB reactions + references.json'))
wckb_metabolites = json.load(open('2021-03-17_APF_Karr et al. metabolite data.json'))
empty_cells = ['NaN', None, numpy.nan, '', ' ', 'null', 'None', 'nan', '[None]', '[\'None\']']    

    
# examine the data from the scraped NIST database
nist_enzymes = []
nist_keqs = []
nist_kms = []
nist_enthalpies = []
for enzyme, value in nist.items():
    if enzyme not in nist_enzymes:
        #print(enzyme)
        nist_enzymes.append(enzyme)
        #print(value['Keq'])
        for key1, values in value['Keq'].items():
            if key1 == 'keqs':      
                #print(key1, values)
                for key in values:
                    if key not in empty_cells and enzyme not in nist_keqs:
                        nist_keqs.append(enzyme)
        for key1, values in value['Km'].items():
            if key1 == 'km values':      
                #print(key1, values)
                for key in values:
                    if key not in empty_cells and enzyme not in nist_kms:
                        nist_kms.append(enzyme)
        for key1, values in value['Enthalpy'].items():
            if key1 == 'enthalpy values':      
                #print(key1, values)
                for key in values:
                    if key not in empty_cells and enzyme not in nist_enthalpies:
                        nist_enthalpies.append(enzyme)

                            
                            
print('NIST enzymes:\t\t', len(nist_enzymes))                      
print('NIST Keqs:\t\t', len(nist_keqs))   
print('NIST Kms:\t\t', len(nist_kms))
print('NIST enthalpies:\t', len(nist_enthalpies))


# examine the data from the downloaded WCKB database
wckb_enzymes = []
wckb_keqs = []
wckb_kinetics = []
wckb_gibbs = []
for enzyme, value in wckb_json.items():
    if enzyme not in wckb_enzymes:
        #print(enzyme)
        wckb_enzymes.append(enzyme)
        for keq in value['keq']:
            if keq not in empty_cells:
                wckb_keqs.append(enzyme)
        for gibbs in value['Gibbs free energy']:
            if gibbs not in empty_cells and enzyme not in wckb_gibbs:
                wckb_gibbs.append(enzyme)
        for forward_kinetics in value['forward kinetics']:
            if forward_kinetics not in empty_cells:
                if enzyme not in wckb_kinetics:
                    wckb_kinetics.append(enzyme)
        for backward_kinetics in value['backward kinetics']:
            if backward_kinetics not in empty_cells:
                if enzyme not in wckb_kinetics:
                    wckb_kinetics.append(enzyme)
                            
print('\n\nWCKB enzymes:\t\t', len(wckb_enzymes))                      
print('WCKB Keqs:\t\t', len(wckb_keqs))   
print('WCKB Kinetics:\t\t', len(wckb_kinetics))
print('WCKB gibbs:\t\t', len(wckb_gibbs))


# contrast the wckb and nist datasets
conjoined_enzymes = []
union_enzymes = {}
thermodynamically_described_enzymes = []
thermodynamically_undescribed_enzymes = []
thermodynamically_described_enzymes_wckb = []
thermodynamically_described_enzymes_nist = []
for enzyme in wckb_enzymes:
    union_enzymes[enzyme.lower()] = {'source 1':'Whole cell'}
    for element in wckb_json[enzyme]['Gibbs free energy']:
        for element2 in wckb_json[enzyme]['keq']:
            if (element or element2) not in empty_cells:
                #print(wckb_json[enzyme]['keq'], '\n', wckb_json[enzyme]['Gibbs free energy'])
                thermodynamically_described_enzymes.append(enzyme.lower())
                thermodynamically_described_enzymes_wckb.append(enzyme)
            else:
                thermodynamically_undescribed_enzymes.append(enzyme.lower())
        
for enzyme2 in nist_enzymes:
    if enzyme2.lower() in union_enzymes.keys():
        union_enzymes[enzyme2.lower()]['source 2'] = 'NIST'
    else:
        union_enzymes[enzyme2.lower()] = {'source 1':'NIST'}
    if enzyme2.lower() not in thermodynamically_described_enzymes:
        thermodynamically_described_enzymes.append(enzyme2.lower())
   
    thermodynamically_described_enzymes_nist.append(enzyme2)
    
#print(enzyme2, '\t', union_enzymes[enzyme2.lower()])
        
described_compounds = []
for enzyme in thermodynamically_described_enzymes_wckb:
    if enzyme.lower() in thermodynamically_described_enzymes:
        for entry in wckb_json[enzyme]['stoichiometry']:
            for molecule in entry:
                try:
                    proper_molecule = wckb_metabolites[molecule['molecule']]['Name']
                    #print(molecule['molecule'])
                    #print(proper_molecule)
                except:
                    print('ignored compound:\t', molecule['molecule'])
                    #proper_molecule = molecule['molecule']
                if proper_molecule not in described_compounds:
                    described_compounds.append(proper_molecule)
        
for enzyme in thermodynamically_described_enzymes_nist:
    reactants_list = []
    products_list = []
    for reaction in nist[enzyme]['reaction']:
        #print('original reaction:\t', reaction) 
        if re.search('(=)', reaction):
            reaction = reaction.split('=')
            #print('reaction:\t', reaction)
            reactants = reaction[0]
            products = reaction[1] 
            #print('reactants:\t', reactants)
            #print('products:\t', products)
            reactants_list = reactants.split('+')
            products_list = products.split('+')
            #print('reactants list:\t', reactants_list)
            #print('products list:\t', products_list)
            for element in reactants_list:
                #print(element)
                element.strip()
                if element not in described_compounds:
                    described_compounds.append(element)
            for element in products_list:
                #print(element)
                element.strip()
                if element not in described_compounds:
                    described_compounds.append(element)
        
        
# contributing total compound information     
'''compound_information = {}    
for compound in described_compounds:
    #print(compound
    compound_information[compound] = { "id":"",
                                      "abbreviation":"",
                                      "aliases":"",
                                      "formula":"",
                                      "mass":'',
                                      "charge":'',
                                      "deltag":'',
                                      "deltagerr":'',
                                      "pka":"",
                                      "pkb":"",
                                      "inchikey":"",
                                      "smiles":"",
                                      "is_cofactor":'',
                                      "is_core":'',
                                      "is_obsolete":'',
                                      "abstract_compound":"",
                                      "comprised_of":"",
                                      "linked_compound":"",
                                      "notes":[],
                                      "source":"" }
    compound_information[compound]['']'''
    
    
    
        
'''for enzyme in wckb_enzymes:
    for enzyme2 in nist_enzymes:
        #print(enzyme)
        if str(enzyme).lower() == str(enzyme2).lower():
            if enzyme not in conjoined_enzymes:
                conjoined_enzymes.append(enzyme)
print('\nenzyme balance:\t', len(wckb_enzymes) + len(nist_enzymes) == len(conjoined_enzymes) + len(union_enzymes), '\n')                
print('Conjoined NIST and WCKB enzymes:', len(conjoined_enzymes), '\n')
for enzyme in conjoined_enzymes:
    print(enzyme)
print('Union of enzymes:\t', len(union_enzymes), '\n')
for enzyme in union_enzymes:
    print(enzyme)'''



print('Quantity of thermodynamically described enzymes:\t', len(thermodynamically_described_enzymes))            
print('Quantity of thermodynamically undescribed enzymes:\t', len(thermodynamically_undescribed_enzymes))        



with open('2021-04-26_APF_ModelSEED formatted compounds.tsv', 'w') as output:
    #output.write("id" + "name" + "abbreviation" + "aliases" + "formula" + "mass" + "charge" + "deltag" + "deltagerr" + "pka" + "pkb" + "inchikey" + "smiles" + "is_cofactor" + "is_core" + "is_obsolete" + "abstract_compound" + "comprised_of" + "linked_compound" + "notes" + "source" + '\n')
    output.write('\t'.join(['NAMES','ID', '\n']))
    number = 0
    for enzyme in described_compounds:
        output.write('\t'.join([enzyme, '%s'%(number)]) + '\n')
        number += 1

In [ ]:
# import libraries and the text files 
import pandas
import json

ids = pandas.read_csv('Unique_ModelSEED_Compound_Aliases.txt', sep = '\t')
names = pandas.read_csv('Unique_ModelSEED_Compound_Names.txt', sep = '\t')
compounds = json.load(open('2021-04-29_APF_nist thermodynamically described compounds for ModelSEED_02.json'))

#display(ids.head(5))
#display(names.head(5))

# parse the compounds through the ModelSEED text files
compound_information = {}
repeated_compounds = []
for compound in compounds:
    for index, modelseed_name in names.iterrows():
        if str(compound).lower() == str(modelseed_name['External ID']).lower():
            if modelseed_name['ModelSEED ID'] not in compound_information.keys():
                for index2, id in ids.iterrows():
                    if id['ModelSEED ID'] == modelseed_name['ModelSEED ID']:
                        if id['Source'] == 'KEGG':
                            print(modelseed_name)
                            #print(modelseed_name['ModelSEED ID'])
                            #print(id['External ID'])
                            compound_information[modelseed_name['ModelSEED ID']] = {'ModelSEED name': modelseed_name['External ID'],
                                                                                    'KEGG ID':id['External ID']}
                            #print(compound_information.keys())

            else:
                repeated_compounds.append(modelseed_name)

compounds_list = []
for compound in compound_information.keys():
    compounds_list.append(compound)
    
print('Identified compounds: ', len(compounds_list))
print('repeated compounds: ', len(repeated_compounds))

with open('2021-05-03_APF_KEGG and ModelSEED_IDs_01', 'w') as output:
    json.dump(compound_information, output, indent = 4)

In [ ]:
# import libraries and the text files 
import pandas
import json

ids = pandas.read_csv('Unique_ModelSEED_Compound_Aliases.txt', sep = '\t')
names = pandas.read_csv('Unique_ModelSEED_Compound_Names.txt', sep = '\t')
compounds = json.load(open('2021-05-09_APF_nist thermodynamically described compounds for ModelSEED_01.json'))

#display(ids.head(5))
#display(names.head(5))

# parse the compounds through the ModelSEED text files
described_compounds = {}
repeated_compounds = []
for compound in compounds:
    for index, modelseed_name in names.iterrows():
        if str(compound).lower() == str(modelseed_name['External ID']).lower():
            if modelseed_name['External ID'] not in described_compounds:
                described_compounds[modelseed_name['External ID']] = modelseed_name['ModelSEED ID']
                
for described_compound, modelseed_id in described_compounds.items():
    for index2, id in ids.iterrows():
        if id['ModelSEED ID'] == modelseed_id:
            if id['Source'] == 'KEGG':
                #print(modelseed_name)
                #print(modelseed_name['ModelSEED ID'])
                #print(id['External ID'])
                compound_information[described_compound] = {'ModelSEED ID': modelseed_id,
                                                            'KEGG ID':id['External ID']}

compounds_list = []
for compound in compound_information.keys():
    compounds_list.append(compound)
    
print('Identified compounds: ', len(compounds_list))
print('repeated compounds: ', len(repeated_compounds))

with open('2021-05-09_APF_KEGG and ModelSEED_IDs_01', 'w') as output:
    json.dump(compound_information, output, indent = 4)

In [ ]:
# import the requisite libraries
import numpy
import json
import pandas
import re

modelseed = json.load(open('compounds.json'))
compounds = json.load(open('2021-05-09_APF_nist thermodynamically described compounds for ModelSEED_01.json'))

empty_cells = ['null', ' null', numpy.nan, 'None', None]

# parse the modelseed database and the list of thermodynamically described compounds
described_compounds = {}
alias_names = []
for compound in compounds:
    if compound not in described_compounds:  
        #print(compound)
        for compound2 in modelseed:
            #print(compound2)
            if compound2['is_obsolete'] == 0:
                if compound2['aliases'] not in empty_cells:
                    for entry in compound2['aliases']:
                        if re.search('Name:', entry):
                            alternative_name_string = re.sub('Name: ', '', entry).split('; ')
                        if re.search('KEGG:', entry):
                            kegg_id = re.sub('KEGG: ', '', entry).split('; ')
                else:
                    alternative_name_string = []

                if str(compound).lower() == str(compound2['name']).lower() and compound not in described_compounds.keys():
                    print('described 1: ', compound)
                    described_compounds[compound] = {'ModelSEED ID': compound2['id'],
                                                     'KEGG ID': kegg_id}
                else:
                    for name in alternative_name_string:
                        try:
                            if re.search(compound, name, re.IGNORECASE) and compound not in described_compounds.keys():
                                print('described 2: ', compound)
                                described_compounds[compound] = {'ModelSEED ID': compound2['id'],
                                                                 'KEGG ID': kegg_id,
                                                                 'alternative names': alternative_name_string}
                        except:
                            break
    
# export the acquired set of described compounds
with open('2021-05-09_APF_described compounds in ModelSEED_01.json', 'w') as output:
    json.dump(described_compounds, output, indent = 5)
    
    

In [ ]:
import pandas
import json
import io

compounds = json.load(open('2021-05-09_APF_nist thermodynamically described compounds for ModelSEED_01.json'))
described_compounds = json.load(open('2021-05-09_APF_described compounds in ModelSEED_01.json'))

described_compounds_list = []
for compound in described_compounds:
    described_compounds_list.append(compound)
    
print('Described compounds: ', len(described_compounds_list))

compounds_list = []
for compound in compounds:
    compounds_list.append(compound)
    
print('Total compounds: ', len(compounds_list))

undescribed_compounds = list(set(compounds_list) - set(described_compounds_list))

print('Undescribed compounds: {}\n'.format(len(undescribed_compounds)), '='*len('Undescribed compounds:'))
for compound in undescribed_compounds:
    print(compound)
    
    
with io.open('2021-05-09_APF_NIST compounds absent from ModelSEED_01.csv', 'w', encoding="utf-8") as output:
    undescribed_compund_csv = pandas.DataFrame(undescribed_compounds)
    undescribed_compund_csv.to_csv(output)

In [ ]:
from ftfy import fix_encoding
import json

nist = json.load(open('2021-05-06_NIST consolidated.json'))
described_compounds = json.load(open('2021-05-09_APF_described compounds in ModelSEED_01.json'))

described_compounds_list = []
for compound in described_compounds:
    described_compounds_list.append(compound)

#print('Described compounds: {}\n'.format(len(described_compounds_list)), described_compounds_list)
    
completely_described_reactions = {}
undescribed_reactions = {}
printed_enzyme = 'no'
for enzyme, information in nist.items():
    described_reactions = []
    for reaction in information['reaction']:
        reaction = fix_encoding(reaction)
        reactants_list = []
        products_list = []
        compounds_list = []
        if re.search('(=)', reaction):
            reaction_split = reaction.split('=')
            reactants_list = reaction_split[0].split(' + ')
            products_list = reaction_split[1].split(' + ')
            compounds = reactants_list + products_list
            for element in compounds:
                #print(element)
                element = element.strip()
                element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element not in compounds_list:
                    compounds_list.append(element)

        #print(compounds_list)
        described = all(compound in described_compounds_list for compound in compounds_list)
        if described:
            described_reactions.append(reaction)

        '''else:
            undescribed_reactions[element] = {'Enzyme': enzyme,
                                              'Reaction': reaction,
                                              'synonym': ''}
        
            continue'''
        
    completely_described_reactions[enzyme] = {'completely described reactions': described_reactions}
    
with open('2021-05-11_APF_completely described reactions for ModelSEED_01.json', 'w') as output:
    json.dump(completely_described_reactions, output, indent = 4)

In [ ]:
import json

def isfloat(number):
    try:
        float(number)
        return True
    except ValueError:
        return False

completely_described_reactions = json.load(open('2021-05-09_APF_completely described reactions for ModelSEED_01.json'))
nist = json.load(open('2021-05-06_NIST consolidated.json'))

described_enzymes_list = []
for enzyme in completely_described_reactions:
    described_enzymes_list.append(enzyme)
    
described_enzyme_data = {}
for enzyme, information in nist.items():
    unique_reactions = []
    temperatures = []
    phs = []
    if enzyme in described_enzymes_list:
        for reaction in information['reaction']:
            if reaction not in unique_reactions:
                unique_reactions.append(reaction)
        
        # calculate the average temperature 
        for temperature in information['experimental temperatures']:
            if isfloat(temperature):
                temperatures.append(float(temperature))
        
        if len(temperatures) > 0:
            average_temperature = sum(temperatures) / len(temperatures)
        else:
            average_temperautre = 'nan'
            
        # calculate the average ph 
        for ph in information['experimental phs']:
            if isfloat(ph):
                phs.append(float(ph))
        
        if len(temperatures) > 0:
            average_ph = sum(phs) / len(phs)
        else:
            average_ph = 'nan'
        

        
    described_enzyme_data[enzyme] = {'unique reactions': unique_reactions,
                                    'average keq': information['Keq']['keq average'],
                                    'average enthalpy': information['Enthalpy']['enthalpy average'],
                                    'average temperature kelvin': average_temperature,
                                    'average pH': average_ph}

with open('2021-05-09_APF_ModelSEED described enzymes_01.json', 'w') as output:
    json.dump(described_enzyme_data, output, indent = 4)

In [ ]:
import json

nist_data = json.load(open('2021-05-09_APF_ModelSEED described enzymes_01.json'))

     
with open('2021-05-10_APF_ModelSEED formatted reactions.tsv', 'w') as output:
    output.write('\t'.join(['ID', 'Names','Equation', 'Equation defintion', '\n']))
    for enzyme, information in nist_data.items():
        for reaction in information['unique reactions']:            
            output.write('\t'.join([information['id'],
                                    enzyme,
                                    equation,
                                    reaction) + '\n')
    output.close()
                                    
        

In [ ]:
from ftfy import fix_encoding
import json

nist_reactions = json.load(open('2021-05-09_APF_ModelSEED described enzymes_01.json'))
nist_compounds = json.load(open('2021-05-09_APF_described compounds in ModelSEED_01.json'))

for enzyme, information in nist_reactions.items():
    for reaction in information['unique reactions']:        
        reaction = fix_encoding(reaction)
        reactants_list = []
        products_list = []
        compounds_list = []
        if re.search('(=)', reaction):
            reaction_split = reaction.split('=')
            reactants_list = reaction_split[0].split(' + ')
            products_list = reaction_split[1].split(' + ')
            reaction_string = ''
            for element in reactants_list:
                #print(element)
                element = element.strip()
                element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element in nist_compounds.keys():
                    element = nist_compounds[element]['ModelSEED ID']
                    reaction_string += element
                    
            product_string = ''
            for element in products_list:
                #print(element)
                element = element.strip()
                element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element in nist_compounds.keys():
                    element = nist_compounds[element]['ModelSEED ID']
                    
                    
with open('2021-05-10_APF_ModelSEED ID described reactions_01.json') as output:
    json.dump( , output, indent = 4)

In [ ]:
from ftfy import fix_encoding
import json
import re

nist_reactions = json.load(open('2021-05-09_APF_ModelSEED described enzymes_01.json'))
nist_compounds = json.load(open('2021-05-09_APF_described compounds in ModelSEED_01.json'))
described_reactions = json.load(open('2021-05-09_APF_nist thermodynamically described reactions for ModelSEED_01.json'))

total_complete_reactions = []
exchanged_reactions = []
to_modelseed_reactions = {}
for enzyme, information in nist_reactions.items():
    #if enzyme in described_reactions.keys():
    #print('\n\n{}\n'.format(enzyme))
    #for modelseed_id in information['ModelSEED ID']:
    complete_reactions = []
    for reaction in information['unique reactions']:
        reaction = fix_encoding(reaction)
        reaction = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', reaction)
        #print('reaction: {}'.format(reaction))

        reactants_list = []
        products_list = []
        compounds_list = []
        if re.search('(=)', reaction):
            reaction_split = reaction.split('=')
            reactants_list = reaction_split[0].split(' + ')
            products_list = reaction_split[1].split(' + ')
            reactants = []
            for element in reactants_list:
                #print(element)
                element = element.strip()
                element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element in nist_compounds.keys():
                    reactants.append(nist_compounds[element]['ModelSEED ID'])

                else:
                    reactants.append(element)

            products = []
            for element in products_list:
                #print(element)
                element = element.strip()
                element = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', element)
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element in nist_compounds.keys():
                    products.append(nist_compounds[element]['ModelSEED ID'])

                else:
                    products.append(element)

            compounds = reactants + products
        
        else:
            print('incorrect reaction: ', reaction)

        if len(reactants) > 1:
            reactants_string = ' + '.join(reactants)
            #print('yes')

        elif len(reactants) == 1:
            reactants_string = reactants[0]

        else:
            print('ERROR: The reaction __ {} __ has zero reactants'.format(reaction))

        #print(products)
        if len(products) > 1:
            products_string = ' + '.join(products)

        elif len(products) == 1:
            products_string = products[0]

        else:
            print('\tERROR: The reaction __ {} __ has zero products'.format(reaction))
            print('\t{}'.format(products_list))

        #print('''reaction_string: {}
        #products_string: {}'''.format(type(reaction_string), type(products_string)))

        final_reaction = reactants_string + ' = ' + products_string
        #print('exchanged reaction: {}'.format(final_reaction))
        exchanged_reactions.append(final_reaction)

        if all('cpd' in compound for compound in compounds):
            complete_reactions.append(final_reaction)

        
        '''for compound, ids in nist_compounds.items():
            if re.search(compound, reaction, re.IGNORECASE):
                if not re.search('\w {}'.format(compound), reaction, re.IGNORECASE):
                reaction = re.sub(compound, ids['ModelSEED ID'], reaction)'''

    to_modelseed_reactions[enzyme] = complete_reactions
    total_complete_reactions += complete_reactions
    
print('Total reactions: {}'.format(len(exchanged_reactions)))                    
print('Parsed and exchaged reactions: {}'.format(len(total_complete_reactions)))
print('Previously complete reactions: {}'.format(len(described_reactions.keys())))
#print(complete_reactions)


'''with open('2021-05-11_APF_NIST to ModelSEED_01.json', 'w') as output:
    json.dump(to_modelseed_reactions, output, indent = 4)'''


'''with open('2021-05-10_APF_ModelSEED ID described reactions_01.json') as output:
    json.dump( , output, indent = 4)'''

In [ ]:
import json
import re

described_reactions = json.load(open('2021-05-11_APF_NIST to ModelSEED, in form_01.json'))

tsv_lines = []
for enzyme, information in described_reactions.items():
    for reaction in information:
        tsv_lines.append('\t'.join([enzyme, reaction]))
        
print('Total reactions: {}'.format(len(tsv_lines)))  
        
tsv_file = open('2021-05-11_APF_NIST to ModelSEED, per reaction, in form_01.tsv', 'w')
tsv_file.write('\t'.join(['ID', 'Equation']) + '\n' + '\n'.join(tsv_lines))
tsv_file.close()    

'''with open('2021-05-10_APF_ModelSEED ID described reactions_01.json') as output:
    json.dump( , output, indent = 4)'''

In [ ]:

from ftfy import fix_encoding
import json
import re

nist_reactions = json.load(open('2021-05-09_APF_ModelSEED described enzymes_01.json'))
nist_compounds = json.load(open('2021-05-09_APF_described compounds in ModelSEED_01.json'))
described_reactions = json.load(open('2021-05-09_APF_nist thermodynamically described reactions for ModelSEED_01.json'))

total_complete_reactions = []
exchanged_reactions = []
to_modelseed_reactions = {}
for enzyme, information in nist_reactions.items():
    #if enzyme in described_reactions.keys():
    #print('\n\n{}\n'.format(enzyme))
    #for modelseed_id in information['ModelSEED ID']:
    complete_reactions = []
    for reaction in information['unique reactions']:
        reaction = fix_encoding(reaction)
        reaction = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', reaction)
        #print('reaction: {}'.format(reaction))

        reactants_list = []
        products_list = []
        compounds_list = []
        if re.search('(=)', reaction):
            reaction_split = reaction.split('=')
            reactants_list = reaction_split[0].split(' + ')
            products_list = reaction_split[1].split(' + ')
            reactants = []
            for element in reactants_list:
                #print(element)
                element = element.strip()
                if re.search('(\d\s|\d\/\d\s)', element):
                    coefficient = re.compile('(\d\s|\d\/\d\s)')
                    coefficient = coefficient.findall(element)[0]
                else:
                    coefficient = ''
                    
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element in nist_compounds.keys():
                    reactants.append(coefficient + nist_compounds[element]['ModelSEED ID'])

                else:
                    reactants.append(element)

            products = []
            for element in products_list:
                #print(element)
                element = element.strip()
                if re.search('(\d\s|\d\/\d\s)', element):
                    coefficient = re.compile('(\d\s|\d\/\d\s)')
                    coefficient = coefficient.findall(element)[0]
                else:
                    coefficient = ''
                    
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element in nist_compounds.keys():
                    products.append(coefficient + nist_compounds[element]['ModelSEED ID'])

                else:
                    products.append(element)

            compounds = reactants + products
            
        
        else:
            print('incorrect reaction: ', reaction)

        if len(reactants) > 1:
            reactants_string = ' + '.join(reactants)
            #print('yes')

        elif len(reactants) == 1:
            reactants_string = reactants[0]

        else:
            print('ERROR: The reaction __ {} __ has zero reactants'.format(reaction))

        #print(products)
        if len(products) > 1:
            products_string = ' + '.join(products)

        elif len(products) == 1:
            products_string = products[0]

        else:
            print('\tERROR: The reaction __ {} __ has zero products'.format(reaction))
            print('\t{}'.format(products_list))

        reaction_string = reactants_string + ' <=> ' + products_string
            
        # arrange the reaction equilibrium arrow and stoichiometric coefficients for ModelSEED                        
        if re.search('((?<!\w)\d\s|\d\/\d\s)', reaction_string):
            reaction_string = re.sub('((?<!\w)\d\s|\d\/\d\s)', r'(\1) ', reaction_string)
            reaction_string = re.sub('(?<=\d)(\s)(?=\))', '', reaction_string)
            
        if re.search('(?<=>|\+)(\s)(?=\w)', reaction_string):
            reaction_string = re.sub('(?<=>|\+)(\s)(?=\w)', ' (1) ', reaction_string)

        if not re.search('^( \d|\()', reaction_string):
            reaction_string = '(1) ' + reaction_string
            
        reaction_string = re.sub('(?<=\w)(\s)(?=<|\+)', '[0] ', reaction_string)
        reaction_string += '[0]'
            
        #print('''reaction_string: {}
        #products_string: {}'''.format(type(reaction_string), type(products_string)))
        #print('exchanged reaction: {}'.format(final_reaction))
        
        
        exchanged_reactions.append(reaction_string)
        if all('cpd' in compound for compound in compounds):
            complete_reactions.append(reaction_string)

        
        '''for compound, ids in nist_compounds.items():
            if re.search(compound, reaction, re.IGNORECASE):
                if not re.search('\w {}'.format(compound), reaction, re.IGNORECASE):
                reaction = re.sub(compound, ids['ModelSEED ID'], reaction)'''

    to_modelseed_reactions[enzyme] = complete_reactions
    total_complete_reactions += complete_reactions
    
print('Total reactions: {}'.format(len(exchanged_reactions)))                    
print('Parsed and exchaged reactions: {}'.format(len(total_complete_reactions)))
print('Previously complete reactions: {}'.format(len(described_reactions.keys())))
#print(complete_reactions)


'''with open('2021-05-11_APF_NIST to ModelSEED, in form_01.json', 'w') as output:
    json.dump(to_modelseed_reactions, output, indent = 4)'''

    
tsv_lines = []
enzyme_id_dictionary = {}
id_number = 0
for enzyme, information in to_modelseed_reactions.items():
    enzyme_id_dictionary[enzyme] = id
    for reaction in information:
        id = 'rxn{}'.format(id_number)
        tsv_lines.append('\t'.join([id, reaction]))
        id_number += 1
    
print('Total reactions: {}'.format(len(tsv_lines)))  
        
tsv_file = open('2021-05-11_APF_NIST to ModelSEED, in form, with ids_01.tsv', 'w')
tsv_file.write('\t'.join(['ID', 'Equation']) + '\n' + '\n'.join(tsv_lines))
tsv_file.close()  
    

'''with open('2021-05-10_APF_ModelSEED ID described reactions_01.json') as output:
    json.dump( , output, indent = 4)'''

The "Add_new_curated_compounds.py" and "Add_new_curated_reactions.py" files were used to evaluate the data contribution. 

## Add_new_curated_compounds.py 

In [1]:
#!/usr/bin/env python
import os, sys, re, copy
from csv import DictReader
from collections import OrderedDict
temp=list();
header=True;

dry_run = True
if("save" in sys.argv):
    dry_run = False

if(len(sys.argv)<2 or os.path.isfile(sys.argv[1]) is False):
    print("Takes one argument, the path to and including reactions file")
    sys.exit()

compounds_file=sys.argv[1]
curation_source = compounds_file.split('\\')[-2]

sys.path.append('../../../Libs/Python')
from BiochemPy import Reactions, Compounds, InChIs

compounds_helper = Compounds()
compounds_dict = compounds_helper.loadCompounds()

names_dict = compounds_helper.loadNames()
searchnames_dict = dict()
all_names_dict = dict()
new_name_count = dict()
for msid in sorted(names_dict):
    for name in names_dict[msid]:
        all_names_dict[name]=1

        searchname = compounds_helper.searchname(name)
        #Avoid redundancy where possible
        if(searchname not in searchnames_dict):
            searchnames_dict[searchname]=msid

original_alias_dict=compounds_helper.loadMSAliases()
source_alias_dict = dict()
all_aliases = dict()
new_alias_count = dict()
for msid in original_alias_dict:
    for source in original_alias_dict[msid]:
        if(source not in source_alias_dict):
            source_alias_dict[source]=dict()

        for alias in original_alias_dict[msid][source]:
            if(alias not in all_aliases):
                all_aliases[alias]=dict()
            all_aliases[alias][msid]=1

            if(alias not in source_alias_dict[source]):
                source_alias_dict[source][alias]=list()
            source_alias_dict[source][alias].append(msid)

for alias in all_aliases:
    all_aliases[alias]=sorted(all_aliases[alias])

Structures_Dict = compounds_helper.loadStructures(["InChI","SMILE"],["KEGG","MetaCyc"])
all_inchis=dict()
all_aliases_InChIs=dict()
for alias in Structures_Dict['InChI']:
    if('Charged' in Structures_Dict['InChI'][alias]):
        for struct in Structures_Dict['InChI'][alias]['Charged']:
            if(struct not in all_inchis):
                all_inchis[struct]=list()
            all_inchis[struct].append(alias)

            if(alias not in all_aliases_InChIs):
                all_aliases_InChIs[alias]=list()
            all_aliases_InChIs[alias].append(struct)
    elif('Original' in Structures_Dict['InChI'][alias]):
        for struct in Structures_Dict['InChI'][alias]['Original']:
            if(struct not in all_inchis):
                all_inchis[struct]=list()
            all_inchis[struct].append(alias)

            if(alias not in all_aliases_InChIs):
                all_aliases_InChIs[alias]=list()
            all_aliases_InChIs[alias].append(struct)

all_smiles=dict()
all_aliases_SMILEs=dict()
for alias in Structures_Dict['SMILE']:
    if('Charged' in Structures_Dict['SMILE'][alias]):
        for struct in Structures_Dict['SMILE'][alias]['Charged']:
            if(struct not in all_smiles):
                all_smiles[struct]=list()
            all_smiles[struct].append(alias)

            if(alias not in all_aliases_SMILEs):
                all_aliases_SMILEs[alias]=list()
            all_aliases_SMILEs[alias].append(struct)
    elif('Original' in Structures_Dict['SMILE'][alias]):
        for struct in Structures_Dict['SMILE'][alias]['Original']:
            if(struct not in all_smiles):
                all_smiles[struct]=list()
            all_smiles[struct].append(alias)

            if(alias not in all_aliases_SMILEs):
                all_aliases_SMILEs[alias]=list()
            all_aliases_SMILEs[alias].append(struct)

#Find last identifier and increment
last_identifier = list(sorted(compounds_dict))[-1]
identifier_count = int(re.sub('^cpd','',last_identifier))

Default_Cpd = OrderedDict({ "id":"cpd00000","name":"null","abbreviation":"null","aliases":"null",
                             "formula":"null","mass":10000000,"charge":0,
                             "deltag":10000000.0,"deltagerr":10000000.0,"pka":"","pkb":"",
                             "inchikey":"","smiles":"",
                             "is_cofactor":0,"is_core":0,"is_obsolete":0,
                             "abstract_compound":"null","comprised_of":"null","linked_compound":"null",
                             "notes":[],"source":"" })
New_Cpd_Count=dict()
Matched_Cpd_Count=dict()
Headers=list()
with open(compounds_file) as fh:
    for line in fh.readlines():
        line=line.strip()
        if(len(Headers)==0):
            Headers=line.split('\t')
            continue

        cpd=dict()
        array=line.split('\t',len(Headers))
        for i in range(len(Headers)):
            cpd[Headers[i].lower()]=array[i]

        (matched_cpd,matched_src)=(None,None)

        #Check that the Structure doesn't already exist, first as InChI, then as SMILE
        if(matched_cpd is None and cpd['inchi'] and cpd['inchi'] in all_inchis):

            msids = dict()
            for alias in all_inchis[cpd['inchi']]:

                #The structures are taken from their sources and the corresponding alias may not yet be registered
                if(alias not in all_aliases):
                    continue

                for msid in all_aliases[alias]:
                    msids[msid]=1

            msids=list(sorted(msids))
            if(len(msids)>0):
                matched_cpd=msids[0]
                matched_src='InChI'

        elif(matched_cpd is None and cpd['smile'] and cpd['smile'] in all_smiles):

            msids = dict()
            for alias in all_smiles[cpd['smile']]:
                #The structures are taken from their sources and the corresponding alias may not yet be registered
                if(alias not in all_aliases):
                    continue

                for msid in all_aliases[alias]:
                    msids[msid]=1

            msids=list(sorted(msids))
            if(len(msids)>0):
                matched_cpd=msids[0]
                matched_src='SMILE'

        #Then check that the Name doesn't already exist
        elif(matched_cpd is None):
            msids=dict()
            for name in cpd['names'].split('|'):
                searchname = compounds_helper.searchname(name)
                if(searchname in searchnames_dict):
                    msids[searchnames_dict[searchname]]=1
            msids=list(sorted(msids))
            if(len(msids)>0):
                matched_cpd=msids[0]
                matched_src='NAMES'

        if(matched_cpd is not None):
            
            if(matched_src not in Matched_Cpd_Count):
                Matched_Cpd_Count[matched_src]=list()
            Matched_Cpd_Count[matched_src].append(matched_cpd)

            #Regardless of match-type, add new names
            #NB at this point, names shouldn't match _anything_ already in the database
            #Names are saved separately as part of the aliases at the end of the script
            for name in cpd['names'].split('|'):
                if(name not in all_names_dict):
                    #Possible for there to be no names in biochemistry?
                    if(matched_cpd not in names_dict):
                        names_dict[matched_cpd]=list()
                    names_dict[matched_cpd].append(name)
                    all_names_dict[name]=1
                    new_name_count[matched_cpd]=1

            #print warning if multiple structures
            if(cpd['inchi'] in all_inchis):
                if(cpd['id'] not in all_aliases_InChIs or cpd['id'] not in all_inchis[cpd['inchi']]):
                    print("Warning: InChI structure for "+cpd['id']+" assigned to different compounds: "+",".join(all_inchis[cpd['inchi']]))

            #print warning if multiple structures
            if(cpd['smile'] in all_smiles):
                if(cpd['ID'] not in all_aliases_SMILEs or cpd['id'] not in all_smiles[cpd['smile']]):
                    print("Warning: SMILE structure for "+cpd['id']+" assigned to different compounds: "+",".join(all_smiles[cpd['smile']]))
                
            #if matching structure or name, add ID to aliases
            if(matched_src != 'ID'):
                if(matched_cpd not in original_alias_dict):
                    original_alias_dict[matched_cpd]=dict()
                if(matched_cpd in original_alias_dict and curation_source not in original_alias_dict[matched_cpd]):
                    original_alias_dict[matched_cpd][curation_source]=list()
                original_alias_dict[matched_cpd][curation_source].append(cpd['id'])
                new_alias_count[matched_cpd]=1

            #Update source type
            compounds_dict[matched_cpd]['source']='Primary Database'

        else:

            #New Compound!
            #Generate new identifier
            identifier_count+=1
            new_identifier = 'cpd'+str(identifier_count)

            new_cpd = copy.deepcopy(Default_Cpd)
            new_cpd['id']=new_identifier
            new_cpd['mass']=float(cpd['mass'])
            new_cpd['charge']=int(cpd['charge'])
            new_cpd['formula']=cpd['formula']

            #Add new identifier with original ID as alias
            original_alias_dict[new_cpd['id']]={curation_source:[cpd['id']]}
            new_alias_count[new_cpd['id']]=1

            #Add new names
            #Names are saved separately as part of the aliases at the end of the script
            for name in cpd['names'].split('|'):
                if(new_cpd['name']=='null'):
                    new_cpd['name']=name
                    new_cpd['abbreviation']=name

                if(name not in all_names_dict):
                    #Possible for there to be no names in biochemistry?
                    if(new_cpd['id'] not in names_dict):
                        names_dict[new_cpd['id']]=list()
                    names_dict[new_cpd['id']].append(name)
                    all_names_dict[name]=1
                    new_name_count[new_cpd['id']]=1

            #If no names at all
            if(new_cpd['name']=='null'):
                new_cpd['name']=cpd['id']
                new_cpd['abbreviation']=cpd['id']

            #Add source type
            new_cpd['source']='Primary Database'
            compounds_dict[new_cpd['id']]=new_cpd
            New_Cpd_Count[new_cpd['id']]=1

#Here, for matches, re-write names and aliases
print("Compounds matched via:")
for src in sorted(Matched_Cpd_Count):
    print("\t"+src+": "+str(len(Matched_Cpd_Count[src])))
print("Saving additional names for "+str(len(new_name_count))+" compounds")
if(dry_run is False):
    compounds_helper.saveNames(names_dict)
print("Saving additional "+curation_source+" aliases for "+str(len(new_alias_count))+" compounds")
if(dry_run is False):
    compounds_helper.saveAliases(original_alias_dict)
print("Saving "+str(len(New_Cpd_Count))+" new compounds from "+curation_source)
if(dry_run is False):
    compounds_helper.saveCompounds(compounds_dict)

#Scripts to run afterwards
#./Merge_Formulas.py
#./Update_Compound_Aliases.py
#../Structures/List_ModelSEED_Structures.py
#../Structures/Update_Compound_Structures_Formulas_Charge.py
#./Rebalance_Reactions.py


Takes one argument, the path to and including reactions file


SystemExit: 

C:\Program Files (x86)\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Add_new_curated_reactions.py

In [ ]:
#!/usr/bin/env python
import os, sys, re, copy

if(len(sys.argv)!=2 or os.path.isfile(sys.argv[1]) is False):
    print("Takes one argument, the path to and including reactions file")
    sys.exit()

reactions_file=sys.argv[1]
curation_source = reactions_file.split('/')[-2]
compound_source = "ModelSEED"

sys.path.append('../../Libs/Python')
from BiochemPy import Reactions, Compounds, InChIs

compounds_helper = Compounds()
compounds_dict = compounds_helper.loadCompounds()
reactions_helper = Reactions()
reactions_dict = reactions_helper.loadReactions()
reactions_codes = reactions_helper.generateCodes(reactions_dict)

Default_Rxn = {"id":"cpd00001","name":"null","abbreviation":"null","aliases":[],
               "code":"null","stoichiometry":"null","equation":"null","definition":"null",
               "reversibility":"=","direction":"=","deltag":"10000000","deltagerr":"10000000",
               "status":"NB","is_obsolete":0,"is_transport":0,
               "abstract_reaction":"null","pathways":"null","ec_numbers":"null",
               "compound_ids":"null","linked_reaction":"null","notes":[],"source":""}

Compounds_Alias_Dict=compounds_helper.loadMSAliases()
Source_Alias_Dict = dict()
for msid in Compounds_Alias_Dict:
    for source in Compounds_Alias_Dict[msid]:
        if(source not in Source_Alias_Dict):
            Source_Alias_Dict[source]=dict()
        for alias in Compounds_Alias_Dict[msid][source]:
            if(alias not in Source_Alias_Dict[source]):
                Source_Alias_Dict[source][alias]=list()
            Source_Alias_Dict[source][alias].append(msid)

original_rxn_alias_dict=reactions_helper.loadMSAliases()
New_Alias_Count=dict()

Names_Dict = reactions_helper.loadNames()
All_Names = dict()
New_Name_Count=dict()
for msid in sorted(Names_Dict):
    for name in Names_Dict[msid]:
        All_Names[name]=1

ECs_Dict = reactions_helper.loadECs()
All_ECs = dict()
New_EC_Count=dict()
for msid in sorted(ECs_Dict):
    for name in ECs_Dict[msid]:
        All_ECs[name]=1

#Find last identifier and increment
last_identifier = list(sorted(reactions_dict))[-1]
identifier_count = int(re.sub('^rxn','',last_identifier))

#If a reaction, after removing cpd redundancy, is empty
#We use a placeholder
Empty_Rxn_ID="rxn14003"

New_Rxn_Count=dict()
Headers=list()
rxns=list()
missing_cpds=dict()
with open(reactions_file) as fh:
    for line in fh.readlines():
        line=line.strip()
        if(len(Headers)==0):
            Headers=line.split('\t')
            continue

        rxn=dict()
        array=line.split('\t',len(Headers))
        for i in range(len(Headers)):
            rxn[Headers[i].upper()]=array[i]

        #Retrieve identifiers from within equation
        #Split based on whitespace, and remove compartment index
        original_cpd_array=rxn['EQUATION'].split(' ')

        new_cpd_array=list()
        for i in range(len(original_cpd_array)):
            #Assumes MS IDs
            if('cpd' not in original_cpd_array[i]):
                continue

            if(re.search('\[[01]\]$',original_cpd_array[i])):
                new_cpd_array.append(re.sub('\[[01]\]$','',original_cpd_array[i]))
            else:
                new_cpd_array.append(original_cpd_array[i])

        all_matched=True
        for new_cpd in new_cpd_array:
            #Assume MS identifiers for time being
            if(new_cpd in compounds_dict):
                msid = new_cpd

#            if(compound_source in Source_Alias_Dict and new_cpd in Source_Alias_Dict[compound_source]):
#                msid = sorted(Source_Alias_Dict[compound_source][new_cpd])[0]

                #set boundary
                bound_msid=msid+"["
                bound_cpd=new_cpd+"["
                esc_cpd = re.escape(bound_cpd)
                
                eqn_array = rxn['EQUATION'].split(" ")
                new_eqn_array = list()
                for entry in eqn_array:
                    entry = re.sub(esc_cpd,bound_msid,entry)
                    new_eqn_array.append(entry)
                rxn['EQUATION'] = " ".join(new_eqn_array)
            else:
                missing_cpds[new_cpd]=1
                all_matched=False

        if(all_matched is False):
            print("Warning: missing "+curation_source+" identifiers for reaction "+rxn['ID']+": "+rxn['EQUATION'])
            continue
        
        rxn_cpds_array = reactions_helper.parseEquation(rxn['EQUATION'])
        adjusted=False
        new_rxn_cpds_array = reactions_helper.removeCpdRedundancy(rxn_cpds_array)
        if(len(new_rxn_cpds_array)!=len(rxn_cpds_array)):
            adjusted=True
        
        rxn_code = reactions_helper.generateCode(new_rxn_cpds_array)
        matched_rxn=None        
        if(len(new_rxn_cpds_array)==0):
            matched_rxn = Empty_Rxn_ID
        else:
            if(rxn_code in reactions_codes):
                matched_rxn = sorted(list(reactions_codes[rxn_code]))[0]

        #Because we adjust for water a posterior
        #We need to include water when matching codes, in case
        if(matched_rxn is None):

            #Find statuses that only have water imbalance
            new_status = reactions_helper.balanceReaction(new_rxn_cpds_array)
            if(new_status == "MI:H:2/O:1" or new_status == "MI:H:-2/O:-1"):
                Water_Adjustment = 1
                if("-1" in new_status):
                    Water_Adjustment = -1

                #Adjust for water
                reactions_helper.adjustCompound(new_rxn_cpds_array,"cpd00001",float(Water_Adjustment))
                rxn_code = reactions_helper.generateCode(new_rxn_cpds_array)
                if(rxn_code in reactions_codes):
                    matched_rxn = sorted(list(reactions_codes[rxn_code]))[0]

        if(matched_rxn is not None):
            #Add Names, EC and Alias
            #Regardless of match-type, add new names
            #NB at this point, names shouldn't match _anything_ already in the database
            #Names are saved separately as part of the aliases at the end of the script
            for name in rxn['NAMES'].split('|'):
                if(name not in All_Names):
                    #Possible for there to be no names in biochemistry?
                    if(matched_rxn not in Names_Dict):
                        Names_Dict[matched_rxn]=list()
                    Names_Dict[matched_rxn].append(name)
                    All_Names[name]=1
                    New_Name_Count[matched_rxn]=1

            if('ECS' in rxn):
                for ec in rxn['ECS'].split('|'):
                    if(ec not in All_ECs):
                        #Possible for there to be no ecs in biochemistry?
                        if(matched_rxn not in ECs_Dict):
                            ECs_Dict[matched_rxn]=list()
                        ECs_Dict[matched_rxn].append(ec)
                        All_ECs[ec]=1
                        New_EC_Count[matched_rxn]=1

            #Add ID to aliases if the match is with a different reaction
            if(matched_rxn not in original_rxn_alias_dict):
                original_rxn_alias_dict[matched_rxn]=dict()
            if(matched_rxn in original_rxn_alias_dict and curation_source not in original_rxn_alias_dict[matched_rxn]):
                original_rxn_alias_dict[matched_rxn][curation_source]=list()
            if(rxn['ID'] not in original_rxn_alias_dict[matched_rxn][curation_source]):
                original_rxn_alias_dict[matched_rxn][curation_source].append(rxn['ID'])
                New_Alias_Count[matched_rxn]=1

            #Update source type
            reactions_dict[matched_rxn]['source']=curation_source

        else:
            
            #New Reaction!
            #Generate new identifier
            identifier_count+=1
            new_identifier = 'rxn'+str(identifier_count)

            new_rxn = copy.deepcopy(Default_Rxn)
            new_rxn['id']=new_identifier

            #Add new identifier with curation source as alias
            original_rxn_alias_dict[new_rxn['id']]={curation_source:[rxn['ID']]}
            New_Alias_Count[new_rxn['id']]=1

            #Add new names
            #Names are saved separately as part of the aliases at the end of the script
            for name in rxn['NAMES'].split('|'):
                if(new_rxn['name']=='null'):
                    new_rxn['name']=name
                    new_rxn['abbreviation']=name

                if(name not in All_Names):
                    #Possible for there to be no names in biochemistry?
                    if(new_rxn['id'] not in Names_Dict):
                        Names_Dict[new_rxn['id']]=list()
                    Names_Dict[new_rxn['id']].append(name)
                    All_Names[name]=1
                    New_Name_Count[new_rxn['id']]=1

            #If no names at all
            if(new_rxn['name']=='null'):
                new_rxn['name']=rxn['ID']
                new_rxn['abbreviation']=rxn['ID']

            #ECs
            if('ECS' in rxn):
                for ec in rxn['ECS'].split('|'):
                    if(ec not in All_ECs):
                        #Possible for there to be no ecs in biochemistry?
                        if(new_rxn['id'] not in ECs_Dict):
                            ECs_Dict[new_rxn['id']]=list()
                        ECs_Dict[new_rxn['id']].append(ec)
                        All_ECs[ec]=1
                        New_EC_Count[new_rxn['id']]=1

            #Add source type
            new_rxn['source']=curation_source

            reactions_dict[new_rxn['id']]=new_rxn
            New_Rxn_Count[new_rxn['id']]=1

            #Rebuild key fields for reaction using parsed equation
            stoichiometry=reactions_helper.buildStoich(new_rxn_cpds_array)
            reactions_helper.rebuildReaction(reactions_dict[new_rxn['id']],stoichiometry)

            #Finally, because several new reactions may share equations
            if(rxn_code not in reactions_codes):
                reactions_codes[rxn_code]=dict()
            reactions_codes[rxn_code][new_rxn['id']]=1
#            print(new_rxn['id'],rxn_code,rxn['ID'])

if(len(missing_cpds)>0):
    print("Missing Compounds: "+"|".join(sorted(missing_cpds)))

#Here, for matches, re-write names, ecs and aliases
if(len(New_EC_Count)>0):
    print("Saving additional ECs for "+str(len(New_EC_Count))+" reactions")
    reactions_helper.saveECs(ECs_Dict)

if(len(New_Name_Count)>0):
    print("Saving additional names for "+str(len(New_Name_Count))+" reactions")
    reactions_helper.saveNames(Names_Dict)

if(len(New_Alias_Count)>0):
    print("Saving additional "+curation_source+" aliases for "+str(len(New_Alias_Count))+" reactions")
    reactions_helper.saveAliases(original_rxn_alias_dict)

if(len(New_Rxn_Count)>0):
    print("Saving "+str(len(New_Rxn_Count))+" new reactions from "+curation_source)
    reactions_helper.saveReactions(reactions_dict)

#Scripts to run afterwards
#../../Biochemistry/Refresh/Rebalance_Reactions.py (very important)
#../../Biochemistry/Refresh/Adjust_Reaction_Protons.py
#../../Biochemistry/Refresh/Adjust_Reaction_Water.py
#../../Biochemistry/Refresh/Merge_Reactions.py (merges may happen because of water)